<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
# <font color='blue'>Data Science Academy</font>
## <font color='blue'>PySpark e Apache Kafka Para Processamento de Dados em Batch e Streaming</font>
## <font color='blue'>Projeto 6</font>
### <font color='blue'>Monitoramento de Criptomoedas em Tempo Real com Kafka, MongoDB e Streamlit</font>
### <font color='blue'>Criação do Consumer Kafka</font>

In [ ]:
!pip install -q pymongo==4.10.1

In [ ]:
# Imports
import json
import time 
from kafka import KafkaConsumer
from json import dumps,loads
from configparser import RawConfigParser
from pymongo import MongoClient

In [ ]:
# Carrega o arquivo de configuração
config_local = RawConfigParser()
config_local.read("dsa_config.conf")

In [ ]:
# Definimos as configurações
server = config_local['Host']['ip']
port = config_local['Host']['port']

In [ ]:
# Cria o consumer
dsa_consumer = KafkaConsumer('dsa-p6-crypto-topic',
                             bootstrap_servers = [f"{server}:{port}"],
                             value_deserializer = lambda x:loads(x.decode('utf-8')))

In [ ]:
# Cria o cliente para o MongoDB
client = MongoClient(config_local['MongoDB']['url'])

In [ ]:
# Usa o cliente para conectar ao banco de dados do MongoDB
db = client[config_local['MongoDB']['database']]

In [ ]:
# Acessa a coleção (que é como uma tabela em um banco de dados relacional)
collection = db[config_local['MongoDB']['collection']]

In [ ]:
# Loop para inserir cada mensagem extraído do tópico Kafka na coleção do MongoDB
# Clique no botão Stop na barra superior quando quiser interromper o consumo do tópico Kafka
for message in dsa_consumer:
    
    # Acessa o valor da mensagem como dicionário
    record = message.value
    
    # Verifica se o campo 'data' existe e é uma lista
    if 'data' in record and isinstance(record['data'], list):
        
        # Itera sobre cada item na lista e remove o campo 'vwap24Hr', aplicando um filtro aos dados
        for item in record['data']:
            
            # Remove o campo, se existir
            item.pop('vwap24Hr', None)  
    
    # Insere o registro filtrado no MongoDB
    collection.insert_one(record)
    
    # Exibe o registro filtrado no console
    print(record)

# Fim